<a href="https://colab.research.google.com/github/faezeh-lbf/Probing-Persian-Language-Models/blob/main/Comparison_tasks_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git
from IPython.display import clear_output 
!pip install -q sentencepiece
!pip install ipywidgets
!pip install bertviz
clear_output()

In [ ]:
!pip install tqdm --upgrade


## Loading the dataset

In [ ]:
from datasets import load_dataset, list_datasets, load_metric
import numpy as np
import random
import torch

BATCH_SIZE = 64
batch_size = BATCH_SIZE
OPTIONS_ORDER = ['بزرگتر','کوچک‌تر']
epochs=8
device = torch.device("cuda")


# size
datasets = load_dataset('json', data_files='/content/size_comp_train.jsonl').shuffle()
dataset_test = load_dataset('json', data_files='/content/size_comp_dev.jsonl').shuffle()

# # num
# datasets = load_dataset('json', data_files='/content/ant-syn_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

# age
# datasets = load_dataset('json', data_files='/content/age_comp_train.jsonl').shuffle()
# dataset_test = load_dataset('json', data_files='/content/age_comp_dev.jsonl').shuffle()

datasets = datasets['train'].train_test_split(test_size=0.1)
datasets['validation'] = datasets['test']
datasets['test'] = dataset_test['train']

# Common

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer


In [ ]:

# @title model checkpoints
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

# roberta-fa-zwnj-base
# TODO: change Model to persian one


In [ ]:
# # @title fix label

# def fix_labels(ex):
#   c=ex['question']['choices']
#   for i in c:
#     if i['label']==0:
#       if i['text']== OPTIONS_ORDER[0]:
#         ex['fixed_label']=ex['label']
#       else:
#         ex['fixed_label']=1-ex['label']
#   ex['question']['stem']=ex['question']['stem'].replace('[MASK]',tokenizer.mask_token)
#   enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
#   ex={**ex,**enc}
#   return ex

# def prepare_dataset(dataset, data_size = -1):
#   if data_size != -1:
#     dataset = dataset.train_test_split(data_size, seed = np.random.randint(1000))['test']
#   encoded_dataset = dataset.map(fix_labels)
#   encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])
#   return encoded_dataset



In [ ]:
# @title fix label & prepare dataset

def fix_labels(ex):
  c=ex['question']['choices']
  options = []
  for i in c:
    options.append(i['text'])
  options = [tokenizer.encode(option)[1] for option in options]
  ex['fixed_label']=ex['label']
  # print(options)
  ex['options'] = torch.tensor(options)
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]',tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  ex={**ex,**enc}
  sol = {}
  cols = ['input_ids', 'attention_mask', 'fixed_label', 'options']
  for k in ex:
    if k in cols:
      sol[k] = ex[k]
  return sol

def prepare_dataset(dataset, data_size = -1):
  if data_size != -1:
    dataset = dataset.train_test_split(data_size, seed = np.random.randint(1000))['test']
  encoded_dataset = dataset.map(fix_labels)
  # print(encoded_dataset[0])
  # raise 1
  encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label', 'options'])
  return encoded_dataset



In [ ]:
# @title train_model function

def train_model(model, train_dataloader, validation_dataloader):
  print("=======================================================================")

  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  optimizer = AdamW(model.parameters(),
                    lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
  total_steps = len(train_dataloader) * epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  criterion = torch.nn.CrossEntropyLoss() 

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 

          b_input_ids =batch['input_ids'].to(device)
          # print(b_input_ids)
          b_input_mask = batch['attention_mask'].to(device)
          # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
          b_labels = batch['fixed_label'].to(device)
          b_options = batch['options'].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # In PyTorch, calling `model` will in turn call the model's `forward` 
          # function and pass down the arguments. The `forward` function is 
          # documented here: 
          # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
          # The results are returned in a results object, documented here:
          # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
          # Specifically, we'll get the loss (because we provided labels) and the
          # "logits"--the model outputs prior to activation.
          # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
          result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels,
                        options = b_options) 
          
          loss = criterion(result, b_labels)
          # print(loss)
          # loss = result.loss
          # logits = result.logits
          # print('-----------------loss------------')
          # print(loss)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          # print(loss.item())
          total_train_loss += loss.item()


          # Perform a backward pass to calculate the gradients.
          # print(loss.requires_grad)
          # loss.requires_grad = True
          # loss = Variable(loss, requires_grad = True)
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          # raise 1

          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)
      # for p in params[0:5]:
      #     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
      # par = list(model.named_parameters())
      # params.append(par)
      # print(params[-4])
      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()
      
      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          # print(batch)
          b_input_ids = batch['input_ids'].to(device)
          b_input_mask = batch['attention_mask'].to(device)
          b_labels = batch['fixed_label'].to(device)
          b_options = batch['options'].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              result = model(ids=b_input_ids, 
                        mask=b_input_mask, 
                        labels=b_labels,
                        options = b_options)

          # Get the loss and "logits" output by the model. The "logits" are the 
          # output values prior to applying an activation function like the 
          # softmax.
          loss = criterion(result, b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = result.detach().cpu().numpy()
          label_ids = b_labels.cpu().numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

  return avg_val_accuracy

In [ ]:
#  @title evaluate_model function

def evaluate_model(model, test_dataloader):
  model.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  # Evaluate data for one epoch
  for batch in test_dataloader:

      b_input_ids = batch['input_ids'].to(device)
      b_input_mask = batch['attention_mask'].to(device)
      b_labels = batch['fixed_label'].to(device)
      b_options= batch['options'].to(device)
      with torch.no_grad():
        result = model(ids=b_input_ids, 
                      mask=b_input_mask, 
                      labels=b_labels,
                      options=b_options)

      # Move logits and labels to CPU
      logits = result.detach().cpu().numpy()
      label_ids = b_labels.cpu().numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)
      

  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
  print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))
  return avg_val_accuracy



In [ ]:
# @title learning curve functions
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW


def create_dataloader(dataset, batch_size=BATCH_SIZE):
  dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler( dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
  return dataloader

def prepare_model(model_class, model_checkpoint):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  base_model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
  # for p in bertmodel.bert.parameters():
  #   p.requires_grad = False
  
  model = model_class(bertmodel, tokenizer)
  return model

  

def learning_curve(model_class, model_checkpoint, datasets, learning_sizes = [64, 128, 256, 512, 1024, 2048, 4096], batch_size=BATCH_SIZE):
  train_dataset = datasets['train']
  val_dataset = datasets['validation']
  val_dataset = prepare_dataset(val_dataset)
  # print(val_dataset)
  validation_dataloader = create_dataloader(val_dataset, batch_size)
  test_dataloader = create_dataloader(prepare_dataset(datasets['test']), batch_size)

  results = {}

  for c in learning_sizes:
    if len(datasets['train']) > c:
      train_dataset2 = prepare_dataset(train_dataset, c)
      train_dataloader = create_dataloader(train_dataset2, batch_size)
      model = prepare_model(model_class, model_checkpoint)
      model.cuda
      train_model(model, train_dataloader, validation_dataloader)
      eval_result = evaluate_model(model, test_dataloader)
      results[c] = eval_result
    else:
      c = len(dataset['train'])
      train_dataset2 = prepare_dataset(train_dataset, c)
      train_dataloader = create_dataloader(train_dataset2, batch_size)
      model = model_class()
      model.cuda
      train_model(model, train_dataloader, validation_dataloader)
      eval_result = evaluate_model(model, test_dataloader)
      results[c] = evel_result
      return results
  return results






In [ ]:

import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Bert

##Zero Shot

In [ ]:
from torch.nn import functional as F
import torch

In [ ]:
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)
k = tokenizer.vocab_size
# bertmodel.cpu()
# print(k)

In [ ]:
# # model.bert.train=False
# for p in model.bert.parameters():
#   p.requires_grad=False
# for t in model.cls.parameters():
#   t.requires_grad=True
#   print(t.i)#.requires_grad)# print(w.name)

In [ ]:
import torch.nn.functional as F
def predic_mask(text, option1, option2, model, tokenizer):
  op1 = tokenizer.encode_plus(option1, return_tensors = "pt")['input_ids'][0][1]
  op2 = tokenizer.encode_plus(option2,  return_tensors = "pt")['input_ids'][0][1]
  text  =text.replace('[MASK]', tokenizer.mask_token)
  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  output = model(**input)
  logits = output.logits
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]
  k = tokenizer.vocab_size
  topk = torch.topk(mask_word, k, dim = 1)
  mask_word_prob = topk[0][0]
  mask_word = topk[1][0]
  op1_index = torch.where(mask_word==op1)
  op2_index = torch.where(mask_word==op2)
  sent1 =  text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op1_index]))
  sent2 = text.replace(tokenizer.mask_token, tokenizer.decode(mask_word[op2_index]))
  if op1_index[0] > op2_index[0]:
    predicted = option2
    # print(sent2)
  else:
    predicted = option1
    # print(sent1)
  # print(predicted)
  return predicted

In [ ]:
# @title evaluate
def evaluate_zero_shot(tokeninzer, model, dataset):
  correct_answers= o1=o2=b=k= 0 
  for obj in dataset:
    # print(obj)
    text = obj['question']['stem']
    option1 = obj['question']['choices'][0]
    option2 = obj['question']['choices'][1]
    predicted = predic_mask(text, option1['text'], option2['text'], tokenizer=tokenizer, model=model)
    label = obj['label']
    # if obj['question']['choices'][label]['text'] == 'جوانتر':
    #   k+=1
    # elif obj['question']['choices'][label]['text'] == 'پیرتر':
    #   b+=1
    # if predicted == 'پیرتر':
    #   print(text.replace('[MASK]' , 'پیرتر'))
    if predicted == option1['text']:
      p_label = option1['label']
      o1 +=1
    elif predicted == option2['text']:
      p_label = option2['label']
      o2+=1
    else:
      print("sth wrong" , predicted)
    result = int(p_label == label) 
    # print(result)
    correct_answers +=result
  # print(o1, o2)
  # print('k , b: ' , k , b)
  # print(correct_answers, len(dataset)- correct_answers)
    # print(label, predicted)
  metric = correct_answers/len(dataset)
  return metric

metric = evaluate_zero_shot(tokenizer, bertmodel, datasets['test'])
print(metric, model_checkpoint)

## Fine-tuning

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch

model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bertmodel = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

bertmodel.cuda()

In [ ]:
# امبرت: ساعت و تلویزیون/////////آینه چراغ گیتار  ساعت دوش موبایل تلویزیون سفال شانه
# توپ سکه پتو خودکار کلاه
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
# model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
# from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# a=tokenizer.encode('بزرگتر')
# for i in a:
#   print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲

In [ ]:
# @title check tokenizer
print('---------')
a=tokenizer.encode('وز')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بزرگ تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('پیر تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('جوان تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('کوچکتر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بزرگتر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('پیر‌تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('جوان‌تر')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('نبود')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')
a=tokenizer.encode('بود')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲
print('---------')

In [ ]:
epochs=4
device = torch.device("cuda")


In [ ]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# import numpy as np
# def load_data(dataset):
# # torch.from_numpy(np.asarray(labels)
#   ids= torch.cat(dataset['input_ids'], dim=0)
#   msk= torch.cat(dataset['attention_mask'], dim=0)
#   lbl= torch.tensor(dataset['fixed_label'])

#   return TensorDataset(ids,msk, lbl)
  

# # b_input_ids = batch['input_ids'].to(device)
# # b_input_mask = batch['attention_mask'].to(device)
# # b_labels = batch['fixed_label'].to(device)
# # encoded_dataset['train']['input_ids'],'attention_mask','fixed_label']

In [ ]:
# @title dataset loader
from torch.utils.data import TensorDataset, random_split

# dataset = TensorDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            encoded_dataset['train'],  # The training samples.
            sampler = RandomSampler( encoded_dataset['train']), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            encoded_dataset['validation'], # The validation samples.
            sampler = RandomSampler(encoded_dataset['validation']), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
        )

In [ ]:
# for step, batch in enumerate(train_dataloader):
#   if step==0:

    # print(batch)

### Bert_MC-MLM

In [ ]:
import torch

class Bert_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer):
    # print("__________-----------------------______________")
    super(Bert_MC_MLM, self).__init__()
    for p in model.bert.parameters():
      p.requres_value=False

    self.model = model
    self.tokenizer = tokenizer
  
  def forward(self, ids, mask, labels, options):
    output = self.model(ids,attention_mask = mask)
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    mask1_word = []
    for i in range(0, len(mask_index)):
      mask1_word.append(logits[i][mask_index[i]])
    mask1_word =  torch.stack(mask1_word)
    indices1 = torch.tensor(options).to(device)
    predicted1 = []
    for i in range(len(mask1_word)):
      x = [mask1_word[i][j] for j in indices1[i]]
      x = torch.stack(x)
      predicted1.append(x)
    predicted1 = torch.stack(predicted1)

    predicted = functional.softmax(predicted1)

    return predicted


In [ ]:

# import torch
# from torch.nn import functional

# class Bert_MC_MLM(torch.nn.Module):
#   def __init__(self, bert_model, tokenizer, options=OPTIONS_ORDER):
#     # for p in bert_model.bert.parameters():
#     #   p.requires_grad = False
#     super(Bert_MC_MLM, self).__init__()
#     self.bert_model = bert_model
#     self.tokenizer = tokenizer
#     self.options = options
  
#   def forward(self, ids, mask, labels):
#     # print("__________-----------------------______________")
#     op1 = tokenizer.encode(self.options[0])[1]
#     op2 = tokenizer.encode(self.options[1])[1]
#     output = self.bert_model(ids,attention_mask = mask)
#     logits = output.logits
    
#     mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

#     # softmax = functional.softmax(logits, dim = -1)
#     # print(softmax[0])
#     mask_word = []
#     # print(len(mask_index))
#     for i in range(0, len(mask_index)):
#       mask_word.append(logits[i][mask_index[i]])
#     mask_word =  torch.stack(mask_word)
#     indices = torch.tensor([op1, op2]).to(device)
#     predicted = torch.index_select(mask_word, 1, indices)
#     # print(predicted[0][0],predicted[0][1])
#     predicted = functional.softmax(predicted)
  
#     return predicted


In [ ]:
# @title CURVE
curve_info= learning_curve(Bert_MC_MLM, model_checkpoint, datasets)


In [ ]:
import matplotlib.pyplot as plt
# curve_info = c
plt.plot(list(curve_info.keys()), list(curve_info.values()), 'ro')
plt.plot(list(curve_info.keys()), list(curve_info.values()), 'r-')
plt.xscale('log')

from google.colab import files
plt.savefig("abc.png")
files.download("abc.png") 


In [ ]:
model = Bert_MC_MLM(bertmodel, tokenizer, options=OPTIONS_ORDER )
model.cuda()
# clear_output()

In [ ]:
# for step, batch in enumerate(train_dataloader):
#   if step==0:
#     print(batch)
#     # print(torch.stack(batch['input_ids']))
#     # print(torch.FloatTensor(np.ndarray(batch['input_ids'])))
# # b_input_ids

In [ ]:
import gc
# del encoded_dataset
# del model
gc.collect()
torch.cuda.empty_cache()

### training loop


In [ ]:
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
acc = train_model(model, train_dataloader, validation_dataloader)

In [ ]:
import torch
c = torch.tensor([[1 , 7 , 5 ,4],[1 , 3 , 3 ,7],[1 , 3 , 7 ,4]])
c2 = torch.tensor([[[1 , 7 , 5 ,4],[1 , 3 , 3 ,9],[1 , 3 , 7 ,4]], [[1 , 7 , 5 ,4],[1 , 3 , 3 ,2],[1 , 3 , 1 ,4]]])
sel = torch.where(c == 7)[1]
c2[0, sel, :]

## evaluation

In [ ]:
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))



#roberta

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer
from torch.nn import functional
import torch

model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
roberta_model = AutoModelForMaskedLM.from_pretrained(model_checkpoint, return_dict = True)

roberta_model.cuda()


In [ ]:
for p in roberta_model.roberta.parameters():
  p.requires_value=False


In [ ]:
epochs=4
device = torch.device("cuda")

In [ ]:
# @title fix label
def fix_labels(ex):
  c=ex['question']['choices']
  for i in c:
    if i['label']==0:
      if i['text']=='کوچکتر':
        ex['fixed_label']=ex['label']
      else:
        ex['fixed_label']=1-ex['label']
  ex['question']['stem']=ex['question']['stem'].replace('[MASK]', tokenizer.mask_token + ' ' + tokenizer.mask_token)
  enc=tokenizer.encode_plus(ex['question']['stem'],truncation=True,padding='max_length', max_length=64)
  
  ex={**ex,**enc}
  return ex

encoded_dataset = datasets.map(fix_labels)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'fixed_label'])


In [ ]:
# @title dataset loader
from torch.utils.data import TensorDataset, random_split

# dataset = TensorDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            encoded_dataset['train'],  # The training samples.
            sampler = RandomSampler( encoded_dataset['train']), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            encoded_dataset['validation'], # The validation samples.
            sampler = RandomSampler(encoded_dataset['validation']), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            encoded_dataset['test'], # The validation samples.
            sampler = SequentialSampler(encoded_dataset['test']), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
        )

### Roberta_MC-MLM

In [ ]:
c = torch.tensor([[1 , 4 , 1 , 4 ], [1 , 4 , 8 , 4 ], [1 , 4 , 4 , 4 ] ])
c2 = torch.tensor([[1 , 4 , 1 , 4 ], [1 , 4 , 8 , 4 ], [1 , 4 , 4 , 4 ] ])
c + c2

In [ ]:
import torch

class Roberta_MC_MLM(torch.nn.Module):
  def __init__(self, model, tokenizer, options=OPTIONS_ORDER):
    # print("__________-----------------------______________")
    super(Roberta_MC_MLM, self).__init__()
    self.model = model
    self.tokenizer = tokenizer
    self.options = [self.tokenizer.encode(option)[1:3] for option in options]
    print (self.options)
  
  def forward(self, ids, mask, labels):
    # print("__________-----------------------______________")
    output = self.model(ids,attention_mask = mask)
    logits = output.logits

    mask_index = torch.where(ids == tokenizer.mask_token_id)[1]

    softmax = functional.softmax(logits, dim = -1)
    mask1_word = []
    mask2_word = []
    for i in range(0, len(mask_index),2):
      mask1_word.append(softmax[i//2][mask_index[i]])
      mask2_word.append(softmax[(i+1)//2][mask_index[i+1]])
    mask1_word =  torch.stack(mask1_word)
    mask2_word =  torch.stack(mask2_word)
    indices1 = torch.tensor([self.options[0][0],self.options[1][0] ]).to(device)
    indices2 = torch.tensor([self.options[0][1], self.options[1][1]]).to(device)
    predicted1 = torch.index_select(mask1_word, 1, indices1)
    predicted2 = torch.index_select(mask2_word, 1, indices2)
    # print(predicted1,predicted2)
    predicted = functional.softmax(predicted1 + predicted2)
  
    return predicted


In [ ]:

model = Roberta_MC_MLM(roberta_model,tokenizer,options=OPTIONS_ORDER)
model.cuda()
# clear_output()

In [ ]:
torch.cuda.empty_cache()

In [ ]:

import random
import numpy as np
from transformers import get_linear_schedule_with_warmup, AdamW
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

optimizer = AdamW(model.parameters(),
                  lr = 2e-1, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss() 

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids =batch['input_ids'].to(device)
        # print(b_input_ids)
        b_input_mask = batch['attention_mask'].to(device)
        # # token_type_ids = torch.stack(batch['token_type_ids']).to(device)
        b_labels = batch['fixed_label'].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        # print(b_input_ids.shape,b_labels.shape,b_input_mask.shape)
        result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels) 
        loss = criterion(result, b_labels)
        # print(loss)
        # loss = result.loss
        # logits = result.logits
        # print('-----------------loss------------')
        # print(loss)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        # loss.requires_grad = True
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        # print(batch)
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['fixed_label'].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(ids=b_input_ids, 
                       mask=b_input_mask, 
                       labels=b_labels)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = criterion(result, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = result.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


In [ ]:

## evaluation

model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
# Evaluate data for one epoch
for batch in test_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    b_labels = batch['fixed_label'].to(device)
    with torch.no_grad():
      result = model(ids=b_input_ids, 
                    mask=b_input_mask, 
                    labels=b_labels)

    # Move logits and labels to CPU
    logits = result.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)
    

# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy on test set: {0:.2f}".format(avg_val_accuracy))


# TODO



*   fix label is dependent on the task ( بود و نبود و بزرگتر کوچک‌تر)
*   

